In [97]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [112]:
import csv
import pandas as pd
import nltk
import numpy as np
import logging
import gensim
import nltk.data

In [114]:
from nltk.tokenize import word_tokenize

In [99]:
from textblob import TextBlob

In [100]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vdelavaud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [101]:
from nltk.corpus import stopwords

In [102]:
import os
#os.chdir("Documents")
os.getcwd()

'C:\\Users\\vdelavaud\\Documents'

In [103]:
# Lecture du fichier 
train = pd.read_csv('classeur1.csv')
train.head()

,id,label,tweet
0,1,0,Le Text Mining est un ensemble de methodes ...
1,2,0,Pour extraire du sens de documents non struct...
2,3,0,Le Text Mining est (quasi) aussi ancien que...
3,5,0,La mesure statistique de la frequence des mot...
4,6,0,Pour la petite histoire l'IBM 704 sorti en 1...


## Faire ressortir les particularités du fichier

In [104]:
# Pour afficher : train[['tweet','nomMethode']].head()

# Nombre de mots pour chaque tweet
train['word_count'] = train['tweet'].apply(lambda x: len(str(x).split(" ")))

# Nombre de caractères pour chaque tweet
train['char_count'] = train['tweet'].str.len() ## this also includes spaces

# Taille moyenne des mots
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['tweet'].apply(lambda x: avg_word(x))

# Nombre de stopwords
stop = stopwords.words('french')
train['stopwords'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))

# Nombre de caractères spéciaux // Pour le cas du Tweet on prends en comtpe le #
#train['hastags'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

# Nombre de numérique
train['numerics'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# Nombre de lettre majuscule (Symbole de colère)
train['upper'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [110]:
train

,id,label,tweet,word_count,char_count,avg_word,stopwords,numerics,upper
0,1,0,Le Text Mining est un ensemble de methodes ...,47,259,5.071429,11,0,0
1,2,0,Pour extraire du sens de documents non struct...,62,392,5.910714,16,0,0
2,3,0,Le Text Mining est (quasi) aussi ancien que...,12,63,5.777778,2,0,0
3,5,0,La mesure statistique de la frequence des mot...,42,263,5.414634,17,0,0
4,6,0,Pour la petite histoire l'IBM 704 sorti en 1...,46,260,4.886364,11,3,1


# Nettoyage du fichier


# Partie 1 : Simple

In [77]:
# Pour afficher un aperçu, mettre train['tweet'].head()

# Tout mettre en minuscule
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Supprimer la ponctuation
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')

# Supprimer les stopwords
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# Recuperer les mots qui reviennent trop fréquemment, les afficher
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()[:5]
# Les supprimer
freq = list(freq.index)
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

# Supprimer les mots rares (Ils sont trop rares pour leur donner du sens, on va donc les supprimer pour obtenir une meilleur compréhension)
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()[-5:]
freq = list(freq.index)
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

# Correcteur d'orthographe
train['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

# Tokenization
TextBlob(train['tweet'][1]).words

# Stemming (Racinisation)
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
st = FrenchStemmer()
train['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Lemmatization
from textblob import Word
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


In [78]:
train['tweet'].head()

0    ensemble methodes technique doutils exploiter ...
1    extraire sappuie technique danalyse linguistiq...
2                     quasi aussi ancien linformatique
3    mesure frequence distribution permet detablir ...
4    petite histoire libm sorti 1955 a epoque cest ...
Name: tweet, dtype: object

# Partie 2 : Complexe

In [133]:
import os
os.getcwd()

'C:\\Users\\vdelavaud\\Documents'

In [134]:
# N-grams (Determiner le sens à partir que la combinaison de n mots)
TextBlob(train['tweet'][0]).ngrams(2)
# On peut aussi l'utiliser sur 3 mots
# TextBlob(train['tweet'][0]).ngrams(3)


# Term frequency (TF)
# Ratio d'un mot en fonction du tweet
# TF = nbr de fois où le terme T apparait dans une ligne / nbr de termes dans la ligne
tf1 = (train['tweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

# Inverse Document Frequency (IDF)
# Un mot ne nous est pas très utile s'il apparaît dans tous les documents
#IDF = log(N/n), where, N is the total number of rows and n is the number of rows in which the word was present.
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['tweet'].str.contains(word)])))

tf1

# Frequence des termes TF-IDF (Term Frequency -Inverse Doc Freq)
# Calcul de la fréquence d'un terme 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(train['tweet'])

train_vect

# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['tweet'])
train_bow
    
# Sentimental Analysis
train['tweet'][:5].apply(lambda x: TextBlob(x).sentiment)
train['sentiment'] = train['tweet'].apply(lambda x: TextBlob(x).sentiment[0] )
train[['tweet','sentiment']].head()

# Mot (qui vectorise d'autres mots)
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)


from gensim.models import KeyedVectors # load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

model['go']

model['away']

(model['go'] + model['away'])/2




array([-0.091342  ,  0.223404  ,  0.58856   , -0.614765  , -0.0838365 ,
        0.5387    , -0.43531   ,  0.349125  ,  0.163308  , -0.28223   ,
        0.53547   ,  0.52797496,  0.096812  ,  0.2879    , -0.0533385 ,
       -0.37232   ,  0.022637  ,  0.574705  , -0.553275  ,  0.385575  ,
        0.565335  ,  0.805405  ,  0.2579965 ,  0.0088565 ,  0.1674905 ,
        0.25543   , -0.571035  , -0.59926   ,  0.422585  , -0.42896   ,
       -0.389065  ,  0.19631   , -0.00933   ,  0.127285  , -0.0487465 ,
        0.381435  , -0.22540998,  0.021299  , -0.1827915 , -0.16490501,
       -0.47944498, -0.431528  , -0.20091   , -0.55665   , -0.32982   ,
       -0.088548  , -0.28038502,  0.219725  ,  0.090537  , -0.67012   ,
        0.0883085 , -0.19332   ,  0.0465725 ,  1.160815  ,  0.0691255 ,
       -2.47895   , -0.33707   ,  0.083195  ,  1.86185   ,  0.283465  ,
       -0.13081   ,  0.927795  , -0.37028   ,  0.1885465 ,  0.66198   ,
        0.505175  ,  0.37748498,  0.1322995 , -0.380375  , -0.02

Nous avons converti la chaîne entière en un vecteur qui peut maintenant être utilisé comme caractéristique dans n'importe quelle technique de modélisation.

Il faut maintenant créer le modèle pour analyser les mails

In [ ]:
# Source
# https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
